# 4. RNN Model All

* RNN model using the entire dataset (not separated into clusters)

# imports

In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm
from datetime import datetime
from IPython.display import display
import tensorflow
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from numpy import array
from numpy import hstack
from tqdm.keras import TqdmCallback

# pd.display options

In [3]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

# Loading the data

In [5]:
df_train = pd.read_pickle("../Data/Prepared_Data/supervised/consumption_all.pkl")
df_train

,meter_id,meter_id_0,meter_id_1,meter_id_2,meter_id_3,meter_id_4,meter_id_5,meter_id_6,meter_id_7,meter_id_8,meter_id_9,meter_id_10,meter_id_11,meter_id_12,date,day_of_year_sin,day_of_year_cos,day_of_week,day_of_month,month_sin,month_cos,is_weekend,energy_cluster,num_bedrooms,detached,flat,semi_detached,terraced,energy n-7,energy n-6,energy n-5,energy n-4,energy n-3,energy n-2,energy n-1,meter_reading
0,0x240e5e22734a44a174b7dabcf1ea00d70d9ec168,0,0,0,0,0,0,0,0,0,0,0,0,1,2017-12-31,1.726064e-02,0.999851,6,31,0.540641,0.841254,1,1,3.0,0.0,0.0,1.0,0.0,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495
1,0x2ce3d582a1316db5bcfe405cbd6070268944778e,0,0,0,0,0,0,0,0,0,0,0,1,0,2017-12-31,1.726064e-02,0.999851,6,31,0.540641,0.841254,1,1,3.0,0.0,0.0,1.0,0.0,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495
2,0x306e6baa9367d3c43fa6ecc2d0054b207d6ef471,0,0,0,0,0,0,0,0,0,0,0,1,1,2017-12-31,1.726064e-02,0.999851,6,31,0.540641,0.841254,1,1,3.0,0.0,0.0,1.0,0.0,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495
3,0x391e6c2169c27de797ccbdf2d623365da28a6d3d,0,0,0,0,0,0,0,0,0,0,1,0,0,2017-12-31,1.726064e-02,0.999851,6,31,0.540641,0.841254,1,1,3.0,0.0,0.0,1.0,0.0,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495
4,0x423fa805ddb0cba9bdb4460f9a78540287eefd0e,0,0,0,0,0,0,0,0,0,0,1,0,1,2017-12-31,1.726064e-02,0.999851,6,31,0.540641,0.841254,1,1,3.0,0.0,0.0,1.0,0.0,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536,11.881495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617640,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,1,0,0,0,0,2017-12-27,-5.176135e-02,0.998659,2,27,0.540641,0.841254,0,0,3.0,1.0,0.0,0.0,0.0,21.999000,26.232000,14.036000,15.990000,15.550500,15.111000,16.506000,18.758500
617641,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,1,0,0,0,0,2017-12-28,-3.451614e-02,0.999404,3,28,0.540641,0.841254,0,0,3.0,1.0,0.0,0.0,0.0,26.232000,14.036000,15.990000,15.550500,15.111000,16.506000,18.758500,21.011000
617642,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,1,0,0,0,0,2017-12-29,-1.726064e-02,0.999851,4,29,0.540641,0.841254,0,0,3.0,1.0,0.0,0.0,0.0,14.036000,15.990000,15.550500,15.111000,16.506000,18.758500,21.011000,19.204000
617643,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,1,0,0,0,0,2017-12-30,-2.449294e-16,1.000000,5,30,0.540641,0.841254,1,0,3.0,1.0,0.0,0.0,0.0,15.990000,15.550500,15.111000,16.506000,18.758500,21.011000,19.204000,17.397000


# Converting to 3 Dimensional so it can be understood by RNN based models
transform input from [samples, features] to [samples, timesteps, features]

In [34]:
### various feature combinations we may want to use ###

# no addInfo and no day_of_year and no energy_cluster
# so contains meter_id's, day of year, day of month,  cyclically encoded month and is_weekend 
dropped_features = ["num_bedrooms","detached","flat","semi_detached","terraced","day_of_year_sin","day_of_year_cos","energy_cluster"] 

In [43]:
#converting unclustered into 3 dimensions
X = df_train.drop({"meter_reading","meter_id","date"}|set(dropped_features),axis=1)
y = df_train["meter_reading"] 

print("X shape",X.shape,"| y shape", y.shape)

display(X)
display(y)

#converting features and labels into np array
X = X.values
y = y.values

X shape (617645, 25) | y shape (617645,)


,meter_id_0,meter_id_1,meter_id_2,meter_id_3,meter_id_4,meter_id_5,meter_id_6,meter_id_7,meter_id_8,meter_id_9,meter_id_10,meter_id_11,meter_id_12,day_of_week,day_of_month,month_sin,month_cos,is_weekend,energy n-7,energy n-6,energy n-5,energy n-4,energy n-3,energy n-2,energy n-1
0,0,0,0,0,0,0,0,0,0,0,0,0,1,6,31,0.540641,0.841254,1,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536
1,0,0,0,0,0,0,0,0,0,0,0,1,0,6,31,0.540641,0.841254,1,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536
2,0,0,0,0,0,0,0,0,0,0,0,1,1,6,31,0.540641,0.841254,1,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536
3,0,0,0,0,0,0,0,0,0,0,1,0,0,6,31,0.540641,0.841254,1,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536
4,0,0,0,0,0,0,0,0,0,0,1,0,1,6,31,0.540641,0.841254,1,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617640,0,1,1,0,0,1,0,1,1,0,0,0,0,2,27,0.540641,0.841254,0,21.999000,26.232000,14.036000,15.990000,15.550500,15.111000,16.506000
617641,0,1,1,0,0,1,0,1,1,0,0,0,0,3,28,0.540641,0.841254,0,26.232000,14.036000,15.990000,15.550500,15.111000,16.506000,18.758500
617642,0,1,1,0,0,1,0,1,1,0,0,0,0,4,29,0.540641,0.841254,0,14.036000,15.990000,15.550500,15.111000,16.506000,18.758500,21.011000
617643,0,1,1,0,0,1,0,1,1,0,0,0,0,5,30,0.540641,0.841254,1,15.990000,15.550500,15.111000,16.506000,18.758500,21.011000,19.204000


0         11.881495
1         11.881495
2         11.881495
3         11.881495
4         11.881495
            ...    
617640    18.758500
617641    21.011000
617642    19.204000
617643    17.397000
617644    15.237000
Name: meter_reading, Length: 617645, dtype: float64

In [45]:
#transforming input from [samples, features] to [samples, timesteps, features] as expected by RNN
X = X.reshape((X.shape[0], X.shape[1], 1))
print("X shape",X.shape)

X shape (617645, 25, 1)


In [46]:
#changing type of array so tensorflow can understand
#X = np.asarray(X).astype('float32')
y = np.asarray(y).astype('float32')

# Now the models

* think I will experiment with LSTMs, CNN-LSTM hybrid, and WaveNet as they all were demonstrated to be successful in HandsOnMl2

# Defining the RNN model
* machine learning mastery chapter 09 & hands on machine learning chapter 15



In [47]:
#parameters
n_features = X.shape[2]
n_steps_in = 7 # using the last 7 inputs
n_steps_out = 1 # 1 output

In [64]:
#defining the model
#2 LSTM layers, Dense output layer
model = Sequential()
model.add(LSTM(2, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
#model.add(LSTM(2, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')

In [65]:
# fit model
model.fit(X, y, epochs=10, verbose=0, callbacks=[TqdmCallback(verbose=2)])

0epoch [00:00, ?epoch/s]

  0%|          | 0.00/19.3k [00:00<?, ?batch/s]

  0%|          | 0.00/19.3k [00:00<?, ?batch/s]

  0%|          | 0.00/19.3k [00:00<?, ?batch/s]

  0%|          | 0.00/19.3k [00:00<?, ?batch/s]

...

  0%|          | 0.00/19.3k [00:00<?, ?batch/s]

  0%|          | 0.00/19.3k [00:00<?, ?batch/s]

  0%|          | 0.00/19.3k [00:00<?, ?batch/s]

  0%|          | 0.00/19.3k [00:00<?, ?batch/s]

  0%|          | 0.00/19.3k [00:00<?, ?batch/s]

In [91]:
### making predictions iteratively ###
columns = pd.date_range(datetime(2018,1,1),datetime(2018,12,31),freq="D")

#getting all the meters
df_preds = pd.DataFrame(columns=["meter_id"])
df_preds["meter_id"] = df_train.meter_id.unique()

features = df_train.drop({"meter_reading","meter_id","date"}|set(dropped_features),axis=1)

# for each meter
for i in range(len(df_preds["meter_id"])):
    X_preds = df_train.loc[df.meter_id == df_preds.meter_id].drop({"meter_reading","meter_id","date"}|set(dropped_features),axis=1)
    display(X_preds)
    #X_preds = 
    # predicting the next 365 days

ValueError: Can only compare identically-labeled Series objects

,meter_id_0,meter_id_1,meter_id_2,meter_id_3,meter_id_4,meter_id_5,meter_id_6,meter_id_7,meter_id_8,meter_id_9,meter_id_10,meter_id_11,meter_id_12,day_of_week,day_of_month,month_sin,month_cos,is_weekend,energy n-7,energy n-6,energy n-5,energy n-4,energy n-3,energy n-2,energy n-1
0,0,0,0,0,0,0,0,0,0,0,0,0,1,6,31,0.540641,0.841254,1,11.805677,11.942665,11.552484,12.231637,12.903379,12.511848,12.101536
